In [ ]:
#| default_exp distill.distillation_callback

In [ ]:
#| include: false
import warnings
warnings.filterwarnings('ignore')
from nbdev.showdoc import *
import torch
import torch.nn.functional as F

%config InlineBackend.figure_format = 'retina'

Knowledge Distillation, sometimes called teacher-student training, is a compression method in which a small (the student) model is trained to mimic the behaviour of a larger (the teacher) model.

The main goal is to reveal what is called the **Dark Knowledge** hidden in the teacher model.

If we take the same [example](https://www.ttic.edu/dl/dark14.pdf) provided by Geoffrey Hinton et al., we have

The main problem of classification is that the output activation function (softmax) will, by design, make a single value really high and squash others.

$$
p_{i}=\frac{\exp \left(z_{i}\right)}{\sum_{j} \exp \left(z_{j}\right)}
$$

With $p_i$ the probability of class $i$, computed from the logits $z$

Here is an example to illustrate this phenomenon:

Let's say that we have trained a model to discriminate between the following 5 classes: [cow, dog, plane, cat, car]

And here is the output of the final layer (the logits) when the model is fed a new input image: 

In [ ]:
logits = torch.tensor([1.3, 3.1, 0.2, 1.9, -0.3])

By judging on the predictions, the model seems confident that the input data is a dog and quite confident that it is definitely not a plane nor a car, with predictions for cow and cat being moderately high.

So the model not only has learned to recognize a dog in the image, but also that a dog is very different from a car and a plane and share similarities with cats and cows. This information is what is called **dark knowledge** !

When passing those predictions through a softmax, we have:

In [ ]:
predictions = F.softmax(logits, dim=-1); predictions

tensor([0.1063, 0.6431, 0.0354, 0.1937, 0.0215])

This is accuenting the differences that we had earlier, discarding some of the dark knowledge acquired earlier. The way to keep this knowledge is to "soften" our softmax outputs, by adding a **temperature** parameter. The higher the temperature, the softer the predictions.

In [ ]:
soft_predictions = F.softmax(logits/3, dim=-1); soft_predictions

tensor([0.1879, 0.3423, 0.1302, 0.2294, 0.1102])

:::{.callout-note}

if the Temperature is equal to 1, then we have regular softmax

:::

When applying Knowledge Distillation, we want to keep the **Dark Knowledge** that the teacher model has acquired during its training but not rely entirely on it. So we combine two losses: 

- The Teacher loss between the softened predictions of the teacher and the softened predictions of the student
- The Classification loss, which is the regular loss between hard labels and hard predictions

The combination between those losses are weighted by an additional parameter α, as:

$$
L_{K D}=\alpha  * \text { CrossEntropy }\left(p_{S}^{\tau}, p_{T}^{\tau}\right)+(1-\alpha) * \text { CrossEntropy }\left(p_{S}, y_{\text {true }}\right)
$$

With $p^{\tau}$ being the softened predictions of the student and teacher

:::{.callout-note}

In practice, the distillation loss will be a [bit different](http://cs230.stanford.edu/files_winter_2018/projects/6940224.pdf) in the implementation

:::

![](imgs/distill.png "Knowledge Distillation")

In [ ]:
#| export
from fastai.vision.all import *

import torch
import torch.nn as nn
import torch.nn.functional as F

from functools import reduce
from typing import Union

/home/HubensN/miniconda3/envs/prune/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/HubensN/miniconda3/envs/prune/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


This can be done with fastai, using the Callback system !

In [ ]:
#| export
class KnowledgeDistillationCallback(Callback):
    def __init__(self, teacher, loss, activations_student=None, activations_teacher=None, weight=0.5):
        self.stored_activation_student, self.stored_activation_teacher  = {}, {}
        store_attr()
        if self.activations_student is not None:
            self.activations_student, self.activations_teacher = listify(activations_student), listify(activations_teacher)
        
    def before_fit(self):
        if self.activations_student and self.activations_teacher : self.register_hooks()
        self.teacher.eval()

    def after_loss(self):
        teacher_pred = self.teacher(self.x)
        new_loss = self.loss(pred=self.pred, teacher_pred=teacher_pred, fm_s=self.stored_activation_student, fm_t=self.stored_activation_teacher)
        self.learn.loss_grad = torch.lerp(self.learn.loss_grad, new_loss, self.weight)
        self.learn.loss = self.learn.loss_grad.clone()
    
    def register_hooks(self):
        self.handles_st, self.handles_t = {}, {}
        for name_st, name_t in zip(self.activations_student, self.activations_teacher):
            self.handles_st[name_st] = get_module_by_name(self.learn, name_st).register_forward_hook(self.get_activation(self.stored_activation_student, name_st))
            self.handles_t[name_t] = get_module_by_name(self.teacher, name_t).register_forward_hook(self.get_activation(self.stored_activation_teacher, name_t))
        
    def get_activation(self, activation, name):
        def hook(model, input, output):
            activation[name] = output
        return hook
    
    def find_hook(self, m):
        save = []
        module_name = type(m).__name__
        for k, v in m._forward_hooks.items():
            save.append((module_name, k, v.__name__))
        return save
    
    def remove_hooks(self, handles):
        for k, v in handles.items():
            handles[k].remove()
    
    def after_fit(self):
        if self.activations_student and self.activations_teacher:
            self.remove_hooks(self.handles_t)
            self.remove_hooks(self.handles_st)

In [ ]:
show_doc(KnowledgeDistillationCallback)

---

### KnowledgeDistillationCallback

>      KnowledgeDistillationCallback (teacher, loss, activations_student=None,
>                                     activations_teacher=None, weight=0.5)

Basic class handling tweaks of the training loop by changing a `Learner` in various events

In [ ]:
#| export
def get_model_layers(model, getLayerRepr=False):
    layers = OrderedDict() if getLayerRepr else []
    def get_layers(net, prefix=[]):
        if hasattr(net, "_modules"):
            for name, layer in net._modules.items():
                if layer is None:
                    continue
                if getLayerRepr:
                    layers[".".join(prefix+[name])] = layer.__repr__()
                else:
                    layers.append(".".join(prefix + [name]))
                get_layers(layer, prefix=prefix+[name])

    get_layers(model)
    return layers


def get_module_by_name(module: Union[torch.Tensor, nn.Module],
                       access_string: str):

    names = access_string.split(sep='.')
    return reduce(getattr, names, module)

The loss function that is used may depend on the use case. For classification, we usually use the one presented above, named `SoftTarget` in fasterai. But for regression cases, we may want to perform regression on the logits directly.

In [ ]:
#| export
def SoftTarget(pred, teacher_pred, T=5, **kwargs):
    return nn.KLDivLoss(reduction='batchmean')(F.log_softmax(pred/T, dim=1), F.softmax(teacher_pred/T, dim=1)) * (T*T)

def Logits(pred, teacher_pred, **kwargs):
    return F.mse_loss(pred, teacher_pred)

def Mutual(pred, teacher_pred, **kwargs):
    return nn.KLDivLoss(reduction='batchmean')(F.log_softmax(pred, dim=1), F.softmax(teacher_pred, dim=1))


def Attention(fm_s, fm_t, p=2, **kwargs):
    return sum([F.mse_loss(F.normalize(fm_s[name_st].pow(p).mean(1),dim=(1,2)), F.normalize(fm_t[name_t].pow(p).mean(1),dim=(1,2))) for name_st, name_t in zip(fm_s, fm_t)])

def ActivationBoundaries(fm_s, fm_t, m=2, **kwargs):
    return sum([((fm_s[name_st] + m).pow(2) * ((fm_s[name_st] > -m) & (fm_t[name_t] <= 0)).float() + (fm_s[name_st] - m).pow(2) * ((fm_s[name_st] <= m) & (fm_t[name_t] > 0)).float()).mean() for name_st, name_t in zip(fm_s, fm_t)])

def FitNet(fm_s, fm_t, **kwargs):
    return sum([F.mse_loss(fm_s[name_st],fm_t[name_t]) for name_st, name_t in zip(fm_s, fm_t)])

def Similarity(fm_s, fm_t, pred, p=2, **kwargs):
    return sum([F.mse_loss(F.normalize(fm_s[name_st].view(fm_s[name_st].size(0), -1) @ fm_s[name_st].view(fm_s[name_st].size(0), -1).t(), p=p, dim=1), F.normalize(fm_t[name_t].view(fm_t[name_t].size(0), -1) @ fm_t[name_t].view(fm_t[name_t].size(0), -1).t(), p=p, dim=1)) for name_st, name_t in zip(fm_s, fm_t)])